# Import

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os, sys
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

from get_dataset import get_training_and_validation


# Load Data

In [ ]:
batch_size = 32
epochs = 30
IMG_HEIGHT = 100
IMG_WIDTH = 100

# make sure to write own classifier-function in next cell
# if num_classes is greater than 2



In [ ]:
# this filter allows all combination with exactly 1 human on it
# artificial masks (photoshop, cg), are not distinguished

# this implicite sets the ['class'] as follows:
#   with_mask >= 1 -> 'mask'
#   with_mask == 0 -> 'no_mask'
#  classifier-attribute can be set to own function for matching data-rows into classes
frame = get_training_and_validation(people_per_img = 1, with_mask=None, no_mask=None, unknown=0,validation_split=0.15 ,test_split=0.05)

train_frame=frame[0]
val_frame=frame[1]
test_frame=frame[2]


print('Filter returned {:d} train images'.format(len(train_frame)))
print(Counter(train_frame['class']))
print('Filter returned {:d} valid images'.format(len(val_frame)))
try:
    print('Filter returned {:d} test images'.format(len(test_frame)))
except:
    print ("Filter returned no test images")
    
num_classes = len(Counter(train_frame['class']))

# Data preparation

In [ ]:
def get_data_generator(image_generator, frame, shuffle=True):
    return image_generator.flow_from_dataframe(frame, directory='./', x_col='Path', y_col='class',
                                                 batch_size = batch_size, shuffle=shuffle, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='categorical')

In [ ]:
# generator for all data
image_generator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=15,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.2
                    )
image_generator_test = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = get_data_generator(image_generator, train_frame)

validation_data_gen = get_data_generator(image_generator_test, val_frame)


In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

# Create the model

In [ ]:
model = Sequential([
    Conv2D(32, 3, padding='same', strides=2,activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', strides=2,activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', strides=2,activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
#model = Sequential()
#model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
##model.add(Dense(num_classes, activation='softmax'))
#model.add(Dense(512, activation='relu'))
#model.add(Dense(1))
#model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']) #TODO: maype 'acc' instead of 'accuracy'

In [ ]:
model.summary()

# Train the model

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = len(train_data_gen),
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps =len(validation_data_gen)
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Data augmentation

## Flip and Shift

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,width_shift_range=.15,
                    height_shift_range=.15,)

train_data_gen = get_data_generator(image_gen, train_frame)

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
# Re-use the same custom plotting function defined and used
# above to visualize the training images
plotImages(augmented_images)

## Rotate

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=15)

train_data_gen = get_data_generator(image_gen, train_frame)

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

## Zoom

In [ ]:
# zoom_range from 0 - 1 where 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [ ]:
train_data_gen = get_data_generator(image_gen, train_frame)

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmented_images)

## Together

In [ ]:
train_data_gen = get_data_generator(image_generator, train_frame)

validation_data_gen = get_data_generator(image_generator_test, val_frame)

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

# Model 2 with Dropout

In [ ]:
model_new = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', strides=2, activation='relu'),
    MaxPooling2D(),
    Dropout(0.1),
    Conv2D(64, 3, padding='same', strides=2, activation='relu'),
    MaxPooling2D(),
    Dropout(0.1),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])


In [ ]:
model_new.compile( optimizer='rmsprop',
                  loss='binary_crossentropy',#tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

# Train2

In [ ]:
checkpoint_filepath = './'

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

In [ ]:
#model_new.fit_generator?

In [ ]:
history = model_new.fit(
    train_data_gen,
    steps_per_epoch=len(train_data_gen),
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps=len(validation_data_gen),
    callbacks=[model_checkpoint_callback,earlyStopping]
)
#model_new.load_weights(checkpoint_filepath)

In [ ]:
epochs=11
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Test

In [ ]:
test_data_gen = get_data_generator(image_generator_test, test_frame, shuffle=False)

In [ ]:
#predict=model.predict(test_data_gen)
#predict

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
model_new.load_weights(checkpoint_filepath)
used_model=model_new

total_test=len(test_frame)

Y_pred = used_model.predict(test_data_gen)#, steps=total_test // batch_size+1)
predict_class = np.argmax(Y_pred, axis=1)
predict_class = predict_class.tolist()

ausgabe=0
if ausgabe:
    print('Predict\n',predict_class)
    print('Klassen\n',test_data_gen.classes)
    classes_table=[int(i=='no_mask') for i in test_frame['class'].tolist()]
    print('Klassen Tabelle\n', classes_table)
    print(test_frame['class'].tolist())


print('Confusion Matrix')
print(confusion_matrix(test_data_gen.classes, predict_class))
print('Classification Report')
target_names = ['Mask','NoMask']
print(classification_report(test_data_gen.classes, predict_class, target_names=target_names))

In [ ]:
from tensorflow.keras.models import save_model, load_model
#used_model.save('saved_model')